__NOTE:__ The networks are not available in this github page. This notebook is provided to make it easier to see the results ran in the thesis.

# WBC

In [7]:
import copy
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
from config import config
import sys
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from lrt_net import BayesianNetwork
import torch.nn.functional as F

In [8]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
lr = config['lr']
class_problem = config["class_problem"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
a_prior = config['a_prior']
SAMPLES = 1



X_original, y_original = load_breast_cancer(return_X_y=True)
n, p = X_original.shape  # need this to get p 

print(n,p)

569 30


In [9]:
# importlib.reload(pf)
for i in range(10):
    net = torch.load(f"trained_nets/wbc/lrt/net{i}", map_location=torch.device('cpu'))
    pf.save_metrics(net, path=f"results/wbc/lrt/net{i}")
    pf.run_path_graph(net, threshold=0.5, save_path=f"path_graphs/wbc/lrt/net{i}", show=False)

{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I9 from I': 0, 'Prob I10 from I': 0, 'Prob I11 from I': 0, 'Prob I12 from I': 0, 'Prob I13 from I': 0, 'Prob I14 from I': 0, 'Prob I15 from I': 0, 'Prob I16 from I': 0, 'Prob I17 from I': 0, 'Prob I18 from I': 0, 'Prob I19 from I': 0, 'Prob I20 from I': 0, 'Prob I21 from I': 0, 'Prob I22 from I': 0, 'Prob I23 from I': 0, 'Prob I24 from I': 0, 'Prob I25 from I': 0, 'Prob I26 from I': 0, 'Prob I27 from I': 0, 'Prob I28 from I': 0, 'Prob I29 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I9 from H1': 0, 'Prob I10 from H1': 0, 'Prob I11 from H1': 0, 'Prob I12 from H1': 0, 'Prob I13 from H1': 0, 'Prob I14 from H1': 0, 'Prob I15 from

In [4]:
import numpy as np

res = {}
data = np.loadtxt(f"results/lrt_class_skip_2_hidden_50_dim_20000_epochs_0.005_lr_wbc_full.txt", delimiter=",")
data_median = np.loadtxt(f"results/lrt_class_skip_2_hidden_50_dim_20000_epochs_0.005_lr_wbc_median.txt", delimiter=",")
res["median_acc_full"] = np.median(data, 0)[0]
res["min_acc_full"] = np.min(data, 0)[0]
res["max_acc_full"] = np.max(data, 0)[0]

res["median_acc_median"] = np.median(data_median, 0)[0]
res["min_acc_median"] = np.min(data_median, 0)[0]
res["max_acc_median"] = np.max(data_median, 0)[0]

res["median_density"] = np.median(data, 0)[1]
res["min_density"] = np.min(data, 0)[1]
res["max_density"] = np.max(data, 0)[1]

res["median_used_weights"] = np.median(data_median, 0)[1]
res["min_used_weights"] = np.min(data_median, 0)[1]
res["max_used_weights"] = np.max(data_median, 0)[1]

res

{'median_acc_full': 0.9482456140350877,
 'min_acc_full': 0.9280701754385964,
 'max_acc_full': 0.9578947368421054,
 'median_acc_median': 0.9149122807017543,
 'min_acc_median': 0.8947368421052632,
 'max_acc_median': 0.9614035087719298,
 'median_density': 0.0013440860202535987,
 'min_density': 0.0010752688394859433,
 'max_density': 0.001612903201021254,
 'median_used_weights': 7.5,
 'min_used_weights': 6.0,
 'max_used_weights': 9.0}

In [10]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/lrt/"
res = {}


res["used_weights"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["used_weights"].append(median_info["used_weights"])

print("USED WEIGHTS:")
m = (res["used_weights"])
print(f"median: {np.median(m)}")
print(f"min: {np.min(m)}")
print(f"max: {np.max(m)}\n")

USED WEIGHTS:
median: 7.5
min: 6
max: 9



In [11]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/lrt/"
res = {}
res["median_depth"] = []
res["max_width"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["median_depth"].append(median_info["avg_path_length"])
    search = True
    i = 0
    while search:
        search = np.sum(median_info["include_inputs"][i]) == 0
        i += 1

    res["max_width"].append(HIDDEN_LAYERS-i+2)

print("Avg. depth")
print(np.median(res["median_depth"]))
print(np.min(res["median_depth"]))
print(np.max(res["median_depth"]))
print("")

print("Max depth")
print(np.median(res["max_width"]))
print(np.min(res["max_width"]))
print(np.max(res["max_width"]))
print("")

Avg. depth
1.0
1.0
1.375

Max depth
1.0
1
2



In [7]:
import numpy as np


import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/wbc/lrt/"
res = {}
for i in range(p):
    res[f"x{i+1}_inclution_rate"] = 0
    
    for n in range(n_nets):
        median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
        #res[d][f"x{i+1}_inclution_rate"] += (median_info["expected_depth_input"][i]>0)
        count = 0
        for j in median_info["include_inputs"]:
            count += j[i]

        res[f"x{i+1}_inclution_rate"] += (count>0)
    res[f"x{i+1}_inclution_rate"] /= n_nets

res

{'x1_inclution_rate': 0.1,
 'x2_inclution_rate': 0.3,
 'x3_inclution_rate': 0.3,
 'x4_inclution_rate': 0.0,
 'x5_inclution_rate': 0.0,
 'x6_inclution_rate': 0.0,
 'x7_inclution_rate': 0.4,
 'x8_inclution_rate': 1.0,
 'x9_inclution_rate': 0.0,
 'x10_inclution_rate': 0.0,
 'x11_inclution_rate': 0.0,
 'x12_inclution_rate': 0.0,
 'x13_inclution_rate': 0.0,
 'x14_inclution_rate': 0.0,
 'x15_inclution_rate': 0.0,
 'x16_inclution_rate': 0.0,
 'x17_inclution_rate': 0.0,
 'x18_inclution_rate': 0.0,
 'x19_inclution_rate': 0.0,
 'x20_inclution_rate': 0.0,
 'x21_inclution_rate': 1.0,
 'x22_inclution_rate': 0.7,
 'x23_inclution_rate': 1.0,
 'x24_inclution_rate': 1.0,
 'x25_inclution_rate': 0.0,
 'x26_inclution_rate': 0.0,
 'x27_inclution_rate': 0.2,
 'x28_inclution_rate': 1.0,
 'x29_inclution_rate': 0.0,
 'x30_inclution_rate': 0.0}

# Abalone

In [1]:
import copy
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import sys
import plot_functions as pf
import pipeline_functions as pip_func
sys.path.append('networks')
from lrt_net import BayesianNetwork
import torch.nn.functional as F

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
lr = config['lr']
class_problem = config["class_problem"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
SAMPLES = 1




#---------DATA------------
df = pd.read_csv("abalone.csv")
df = pd.get_dummies(df, drop_first=True)*1
#df = df.drop(["Sex"])
X_original = df.loc[:,df.columns != "Rings"].values
y_original = df.loc[:,df.columns == "Rings"].values.T[0]
n, p = X_original.shape  # need this to get p 

print(n,p)

4177 9


In [4]:
# importlib.reload(pf)
for i in range(10):
    net = torch.load(f"trained_nets/abalone/lrt/net{i}", map_location=torch.device('cpu'))
    pf.save_metrics(net, path=f"results/abalone/lrt/net{i}")
    pf.run_path_graph(net, threshold=0.5, save_path=f"path_graphs/abalone/lrt/net{i}", show=False)

{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I0 from H2': 0, 'Prob I1 from H2': 0, 'Prob I2 from H2': 0, 'Prob I3 from H2': 0, 'Prob I4 from H2': 0, 'Prob I5 from H2': 0, 'Prob I6 from H2': 0, 'Prob I7 from H2': 0, 'Prob I8 from H2': 0}
H2
{'Prob I0 from I': 0, 'Prob I1 from I': 0, 'Prob I2 from I': 0, 'Prob I3 from I': 0, 'Prob I4 from I': 0, 'Prob I5 from I': 0, 'Prob I6 from I': 0, 'Prob I7 from I': 0, 'Prob I8 from I': 0, 'Prob I0 from H1': 0, 'Prob I1 from H1': 0, 'Prob I2 from H1': 0, 'Prob I3 from H1': 0, 'Prob I4 from H1': 0, 'Prob I5 from H1': 0, 'Prob I6 from H1': 0, 'Prob I7 from H1': 0, 'Prob I8 from H1': 0, 'Prob I0 from H2': 0, '

In [34]:
import numpy as np

res = {}
data = np.loadtxt(f"results/lrt_reg_skip_2_hidden_200_dim_2000_epochs_0.01_lr_abalone_full.txt", delimiter=",")
data_median = np.loadtxt(f"results/lrt_reg_skip_2_hidden_200_dim_2000_epochs_0.01_lr_abalone_median.txt", delimiter=",")
res["median_rmse_full"] = np.median(data, 0)[0]
res["min_rmse_full"] = np.min(data, 0)[0]
res["max_rmse_full"] = np.max(data, 0)[0]

res["median_rmse_median"] = np.median(data_median, 0)[0]
res["min_rmse_median"] = np.min(data_median, 0)[0]
res["max_rmse_median"] = np.max(data_median, 0)[0]

res["median_density"] = np.median(data, 0)[1]
res["min_density"] = np.min(data, 0)[1]
res["max_density"] = np.max(data, 0)[1]

res["median_used_weights"] = np.median(data_median, 0)[1]
res["min_used_weights"] = np.min(data_median, 0)[1]
res["max_used_weights"] = np.max(data_median, 0)[1]

res["median_r2_full"] = np.median(data, 0)[2]
res["min_r2_full"] = np.min(data, 0)[2]
res["max_r2_full"] = np.max(data, 0)[2]

res["median_r2_median"] = np.median(data_median, 0)[2]
res["min_r2_median"] = np.min(data_median, 0)[2]
res["max_r2_median"] = np.max(data_median, 0)[2]

res["median_corr_full"] = np.median(np.sqrt(data), 0)[2]
res["min_corr_full"] = np.min(np.sqrt(data), 0)[2]
res["max_corr_full"] = np.max(np.sqrt(data), 0)[2]

res["median_corr_median"] = np.median(np.sqrt(data_median), 0)[2]
res["min_corr_median"] = np.min(np.sqrt(data_median), 0)[2]
res["max_corr_median"] = np.max(np.sqrt(data_median), 0)[2]

res

{'median_rmse_full': 2.115133285522461,
 'min_rmse_full': 2.0700676441192627,
 'max_rmse_full': 2.170462131500244,
 'median_rmse_median': 2.109241008758545,
 'min_rmse_median': 2.067035436630249,
 'max_rmse_median': 2.1697261333465576,
 'median_density': 0.0007989225850906223,
 'min_density': 0.000525006209500134,
 'max_density': 0.0028304685838520527,
 'median_used_weights': 35.0,
 'min_used_weights': 23.0,
 'max_used_weights': 124.0,
 'median_r2_full': 0.6021074056625366,
 'min_r2_full': 0.5809487104415894,
 'max_r2_full': 0.618885338306427,
 'median_r2_median': 0.6043158769607544,
 'min_r2_median': 0.5807948112487793,
 'max_r2_median': 0.6199920177459717,
 'median_corr_full': 0.7759556546884868,
 'min_corr_full': 0.7621999150102218,
 'max_corr_full': 0.7866926580987184,
 'median_corr_median': 0.7773772189646041,
 'min_corr_median': 0.7620989510875732,
 'max_corr_median': 0.7873957186484898}

In [5]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/lrt/"
res = {}


res["used_weights"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["used_weights"].append(median_info["used_weights"])

print("USED WEIGHTS:")
m = (res["used_weights"])
print(f"median: {np.median(m)}")
print(f"min: {np.min(m)}")
print(f"max: {np.max(m)}\n")

USED WEIGHTS:
median: 35.0
min: 23
max: 124



In [6]:
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/lrt/"
res = {}
res["median_depth"] = []
res["max_width"] = []

for n in range(n_nets):
    median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
    res["median_depth"].append(median_info["avg_path_length"])
    search = True
    i = 0
    while search:
        search = np.sum(median_info["include_inputs"][i]) == 0
        i += 1

    res["max_width"].append(HIDDEN_LAYERS-i+2)

print("Avg. depth")
print(np.median(res["median_depth"]))
print(np.min(res["median_depth"]))
print(np.max(res["median_depth"]))
print("")

print("Max depth")
print(np.median(res["max_width"]))
print(np.min(res["max_width"]))
print(np.max(res["max_width"]))
print("")

Avg. depth
2.506756756756757
1.8823529411764706
2.7435897435897436

Max depth
3.0
3
3



In [39]:
import numpy as np


import os
import sys
current_dir = os.getcwd()
sys.path.append('layers')
from config import config

# Parameters from config file
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
dim = config['hidden_dim']
n_nets = config['n_nets']

path = "results/abalone/lrt/"
res = {}
for i in range(p):
    res[f"x{i+1}_inclution_rate"] = 0
    
    for n in range(n_nets):
        median_info = np.load(path+f"net{n}_median.npy", allow_pickle=True).item()
        #res[d][f"x{i+1}_inclution_rate"] += (median_info["expected_depth_input"][i]>0)
        count = 0
        for j in median_info["include_inputs"]:
            count += j[i]

        res[f"x{i+1}_inclution_rate"] += (count>0)
    res[f"x{i+1}_inclution_rate"] /= n_nets

res

{'x1_inclution_rate': 0.9,
 'x2_inclution_rate': 0.7,
 'x3_inclution_rate': 0.2,
 'x4_inclution_rate': 1.0,
 'x5_inclution_rate': 1.0,
 'x6_inclution_rate': 0.4,
 'x7_inclution_rate': 1.0,
 'x8_inclution_rate': 1.0,
 'x9_inclution_rate': 0.2}

In [40]:
df.loc[:,df.columns != "Rings"].columns

Index(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight', 'Sex_I', 'Sex_M'],
      dtype='object')